In [1]:
import numpy as np
import bokeh
from bokeh.plotting import show
import matplotlib.pyplot as plt
import flowkit as fk
import os
import pandas as pd
from bokeh.io import export_png
from bokeh.plotting import figure, output_file, save
import warnings

In [2]:
bokeh.io.output_notebook()
%matplotlib inline

_ = plt.ioff()

Loading BokehJS ...

## ConvexGating strategy as flowkit strategy

### Functions

In [3]:
def make_gate_edges_object_string(cluster,hierarchy):
    return 'cluster_' + cluster + '_gate_edges_hierarchy_' + str(hierarchy) + '.csv'

def get_flowkit_gating_strategy(base_path,cluster):
    #base_path: location of ConvexGating output files
    #cluster: cluster identifier in ConvexGating run

    # initialize flowkit gating strategy
    #key_cl = next((key for key, value in cg_meta_info['clusterkeys'].items() if value == cluster), None)
    #print(key_cl)
    g_strat = fk.GatingStrategy()

    # get gating depth
    perf_summary_df = pd.read_csv(os.path.join(base_path,'performance_summary.csv'))
    marker_csv = pd.read_csv(os.path.join(base_path,'marker_summary.csv'))
    perf_summary_df['cluster'] = perf_summary_df['cluster'].astype(str)
    max_hierarchy = perf_summary_df[(perf_summary_df['cluster'] == str(cluster))]['hierarchy'].values[0] 

    #loop over hierarchies
    for hierarchy in range(1,max_hierarchy+1):
        hierarchy_string = os.path.join(base_path,'cluster_' + cluster,make_gate_edges_object_string(cluster,hierarchy))
        vertices_hierarchy = pd.read_csv(hierarchy_string,index_col=0)
        vertices = list(zip(vertices_hierarchy['x_coordinate'], vertices_hierarchy['y_coordinate']))
        try:
            markers = list(marker_csv[(marker_csv['cluster'] == int(cluster)) & ((marker_csv['hierarchy'] == hierarchy))]['marker'])
            if len(markers) == 0:
                markers = list(marker_csv[(marker_csv['cluster'] == cluster) & ((marker_csv['hierarchy'] == hierarchy))]['marker'])
        except:
            markers = list(marker_csv[(marker_csv['cluster'] == cluster) & ((marker_csv['hierarchy'] == hierarchy))]['marker'])
        dim_a = fk.Dimension(markers[0])
        dim_b = fk.Dimension(markers[1])
        poly_gate = fk.gates.PolygonGate('hierarchy_' + str(hierarchy),dimensions= [dim_a,dim_b],vertices=vertices)
        if hierarchy == 1:
            gate_path_tuple = ('root',)
        else:
            gate_path_tuple = tuple(['root'] + ['hierarchy_' + str(i) for i in range(1, hierarchy)])
        g_strat.add_gate(poly_gate, gate_path = gate_path_tuple) 
    return g_strat

def add_flowkit_gating_strategies(base_path):
    meta_info_path = os.path.join(base_path,"meta_info.npy")
    flowkit_path = os.path.join(base_path,'flowkit_export')
    if not os.path.exists(flowkit_path):
        os.mkdir(flowkit_path)
    meta_info = np.load(meta_info_path,allow_pickle='TRUE').item()
    flowkit_strategies_dict = {}
    for cluster in list(meta_info['clusterkeys'].values()):
        flowkit_strategies_dict[cluster] = get_flowkit_gating_strategy(base_path,cluster)
    np.save(os.path.join(flowkit_path, "flowkit_strategies.npy"), flowkit_strategies_dict)

def export_strategies(base_path):
    flowkit_path = os.path.join(base_path,'flowkit_export')
    flowkit_fcs_path = os.path.join(flowkit_path,'fcs_samples')
    flowkit_wsp_path = os.path.join(flowkit_path,'wsp_files')
    meta_info_path = os.path.join(base_path,"meta_info.npy")
    meta_info = np.load(meta_info_path,allow_pickle='TRUE').item()
    flowkit_fcs_path = os.path.join(flowkit_path,'fcs_samples')
    flowkit_wsp_path = os.path.join(flowkit_path,'wsp_files')
    if not os.path.exists(flowkit_fcs_path):
            os.mkdir(flowkit_fcs_path)
    if not os.path.exists(flowkit_wsp_path):
            os.mkdir(flowkit_wsp_path)
    flowkit_strategies = np.load(os.path.join(flowkit_path, "flowkit_strategies.npy"),allow_pickle = True).item()
    for key in meta_info['clusterkeys'].keys():
        cluster = meta_info['clusterkeys'][key]
        sample_data = meta_info['general_summary'][key].filter(regex=r'^(?!gate_|new_gate_|final_gate_|true_label)')
        sample_label = meta_info['general_summary'][key][['true_label']]
        sample = fk.Sample(sample_data,sample_id = str(cluster))
        flowkit_out = flowkit_strategies[cluster].gate_sample(sample)
        sample.export(os.path.join(flowkit_fcs_path, 'fcs_cluster_' + cluster + '.fcs'),source='raw')
        fk._utils.wsp_utils.export_flowjo_wsp(flowkit_strategies[cluster],
                                              'wsp_cluster_' + str(cluster),
                                              [sample],
                                              os.path.join(flowkit_wsp_path,'wsp_cluster_' + str(cluster) + '.wsp'))
def get_bokeh_plot(base_path):
    meta_info_path = os.path.join(base_path,"meta_info.npy")
    meta_info = np.load(meta_info_path,allow_pickle='TRUE').item()
    flowkit_path = os.path.join(base_path,'flowkit_export')
    flowkit_fcs_path = os.path.join(flowkit_path,'fcs_samples')
    flowkit_wsp_path = os.path.join(flowkit_path,'wsp_files')
    flowkit_visualize_path = os.path.join(flowkit_path,'plots')
    if not os.path.exists(flowkit_fcs_path):
                os.mkdir(flowkit_fcs_path)
    if not os.path.exists(flowkit_visualize_path):
                os.mkdir(flowkit_visualize_path)
    plot_dict_all = {}
    for key in meta_info['clusterkeys']:
        cluster = meta_info['clusterkeys'][key]
        wsp_path = os.path.join(flowkit_wsp_path,'wsp_cluster_' + str(cluster) + '.wsp')
        fcs_path = os.path.join(flowkit_fcs_path,'fcs_cluster_' + str(cluster) + '.fcs')
        sample = fk.Sample(fcs_path,sample_id = cluster)
        wsp = fk.Workspace(wsp_path, sample)
        wsp.analyze_samples()
        #plot_dict = {}
        cluster_plot_path = os.path.join(flowkit_visualize_path,cluster)
        if not os.path.exists(cluster_plot_path):
                os.mkdir(cluster_plot_path)
        for h_string in list(wsp.get_analysis_report()['gate_name'].values):
            p = wsp.plot_gate(
                cluster,
                h_string,
                subsample_count=10e8)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                save(p,os.path.join(os.path.join(cluster_plot_path,h_string)) + '.html')

def convert_output(base_path):
    add_flowkit_gating_strategies(base_path)
    export_strategies(base_path)
    get_bokeh_plot(base_path)

### Apply

In [4]:
base_path = os.path.join("C:\\Users","vfriedrich","projects","gating","tests","add_hull_try_out","GL02_oetjen_strategies")
convert_output(base_path)

##### Package versions

In [14]:
import pkg_resources
for package in pkg_resources.working_set:
    print(f"{package.key}=={package.version}")

babel==2.13.1
qtpy==2.4.1
sqlalchemy==2.0.23
send2trash==1.8.2
anyio==4.1.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.1.0
beautifulsoup4==4.12.2
bleach==6.1.0
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
colorama==0.4.6
comm==0.2.0
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.0
fqdn==1.5.1
greenlet==3.0.1
idna==3.6
importlib-metadata==7.0.0
ipykernel==6.27.1
ipython==8.18.1
ipython-genutils==0.2.0
ipython-sql==0.5.0
ipywidgets==8.1.1
isoduration==20.11.0
jedi==0.19.1
json5==0.9.14
jsonpointer==2.4
jsonschema==4.20.0
jsonschema-specifications==2023.11.2
jupyter==1.0.0
jupyter-client==8.6.0
jupyter-console==6.6.3
jupyter-core==5.5.0
jupyter-events==0.9.0
jupyter-lsp==2.2.1
jupyter-server==2.11.2
jupyter-server-terminals==0.4.4
jupyterlab==4.0.9
jupyterlab-pygments==0.3.0
jupyterlab-server==2.25.2
jupyterlab-widgets==3.0.9
matplotlib-inline==0.1.6
